In [ ]:
# 成功版本,可持續使用
import instaloader
from datetime import datetime, timedelta
import time
import random
from fake_useragent import UserAgent
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import http.cookiejar as cookielib

# 创建ChromeDriver实例
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("D:/chromedriver-win64/chromedriver-win64/chromedriver")
driver = webdriver.Chrome(options=chrome_options)

# 导航到Instagram登录页面
driver.get("https://www.instagram.com/accounts/login/")
time.sleep(3)  # 这里可以根据实际情况调整等待时间

# 输入用户名和密码,并提交登录
your_username = "wesleywu70"
your_password = "@13267738117"
username_input = driver.find_element('name', 'username')
password_input = driver.find_element('name', 'password')
username_input.send_keys(your_username)
password_input.send_keys(your_password)
password_input.send_keys(Keys.RETURN)

time.sleep(15)

# 获取登录后的cookies
cookies = driver.get_cookies()

# 创建Instaloader实例并加载cookies 
L = instaloader.Instaloader()
ua = UserAgent()
L.context.user_agent = ua.random

# 将cookies保存到文件中
cookie_file = "cookies.txt"
with open(cookie_file, 'w') as f:
    for cookie in cookies:
        f.write(f"{cookie['name']}={cookie['value']}\n")

# 创建InstaloaderContext实例并手动设置cookies
context = instaloader.InstaloaderContext()
for cookie in cookies:
    #context.cookies.set(cookie['name'], cookie['value'])
    context._session.cookies.set(cookie['name'], cookie['value'])
    # 设置cookies到InstaloaderContext实例中

# 设置随机用户代理
ua = UserAgent()
context.user_agent = ua.random

# 指定目标Instagram账户
target_account = "kfchk"

# 设置时间范围
start_date_str = "2023-11-1"
end_date_str = "2023-12-1"
start_time = datetime.strptime(start_date_str, "%Y-%m-%d")
end_time = datetime.strptime(end_date_str, "%Y-%m-%d")

# Create a profile object for the target account
try:
    profile = instaloader.Profile.from_username(context, target_account)
except instaloader.exceptions.ProfileNotExistsException:
    print(f"Profile {target_account} does not exist.")
    exit()

# Set initial delay and retry count
delay = 2
max_retries = 3

# Loop through the overall time range in 10-day intervals
current_start = start_time
while current_start < end_time:
    current_end = current_start + timedelta(days=10)
    if current_end > end_time:
        current_end = end_time

    print(f"Retrieving data from {current_start.date()} to {current_end.date()}")

    data = []

    # Iterate over the posts of the target account
    for post in profile.get_posts():
        # Check if the post is within the current 10-day interval
        if current_start <= post.date <= current_end:
            # Construct the post URL
            post_url = f"https://www.instagram.com/p/{post.shortcode}/"

            retry_count = 0
            while retry_count < max_retries:
                try:
                    # Retrieve the comments for the post
                    post_comments = post.get_comments()

                    # Iterate over the comments
                    for comment in post_comments:
                        # Extract comment details
                        comment_text = comment.text
                        comment_author = comment.owner.username
                        comment_date = comment.created_at_utc

                        # Append data to the list
                        data.append({
                            "Post URL": post_url,
                            "Post Date": post.date,
                            "Comment": comment_text,
                            "Comment Author": comment_author,
                            "Comment Date": comment_date
                        })

                    # Add a random delay between each comment request
                    time.sleep(random.uniform(5, 20))
                    break  # Break out of the retry loop if successful

                except instaloader.exceptions.ConnectionException:
                    print(f"Connection error occurred. Retrying in {delay} seconds...")
                    time.sleep(delay)
                    delay *= 3  # Exponential backoff
                    retry_count += 1

                except instaloader.exceptions.TooManyRequestsException:
                    print(f"Too many requests. Waiting for {delay} seconds before retrying...")
                    time.sleep(delay)
                    delay *= 3  # Exponential backoff
                    retry_count += 1

            # Add a random delay between each post request
            time.sleep(random.uniform(5, 20))

    # Create a DataFrame from the data listcontext.context.session.cookies.set(cookie['name'], cookie['value'])
    df = pd.DataFrame(data)

    # Save the DataFrame to a CSV file
    file_name = f"data_{current_start.date()}_{current_end.date()}.csv"
    df.to_csv(file_name, index=False)

    print(f"Data saved to {file_name}")

    # Move to the next 10-day interval
    current_start = current_end + timedelta(days=1)

In [ ]:
#合并每一個日期成excel
import pandas as pd
import glob

# 获取当前目录下所有以"data_"开头的CSV文件
file_pattern = "data_*.csv"
csv_files = glob.glob(file_pattern)

# 创建一个空的DataFrame来存储所有数据
all_data = pd.DataFrame()

# 逐个读取CSV文件并合并数据
for csv_file in csv_files:
    df = pd.read_csv(csv_file, sep=',', header=0, names=['Post URL', 'Post Date', 'Comment', 'Comment Author', 'Comment Date'])
    all_data = pd.concat([all_data, df], ignore_index=True)

# 保存合并后的数据到一个新的XLSX文件中
merged_file_name = "merged_data.xlsx"
all_data.to_excel(merged_file_name, index=False)

print(f"All CSV files merged and saved to {merged_file_name}")